In [1]:
import numpy as np
from scipy.stats.mstats import mode
from copy import deepcopy
import pandas as pd
import time
%matplotlib inline
from helper import *

In [2]:
df = pd.read_csv('data/train_linear.csv', skipinitialspace=True)

### huge linear model

In [3]:
# sklearn functions used for the linear regression model
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [4]:
# convert to time buckets
# extract time feature (5minute features!)
df['btime'] = (df['timestamp'] % (100 * 100)) // 100 * 100 + ((df['timestamp'] % 100) // 5) * 5

In [5]:
df.head()

,timestamp,SID,count,btime
0,10000,1,0,0
1,10000,2,0,0
2,10000,3,0,0
3,10000,4,0,0
4,10000,5,0,0


In [6]:
# use only categorical features first, encode time and SID!
categoricalFeat = df[['btime', 'SID']]

# encode btime as categorical features
encoder = OneHotEncoder(sparse=True) 
if len(categoricalFeat.columns) == 1:
    categoricals_encoded = encoder.fit_transform(categoricalFeat.reshape(-1, 1))
else:
    categoricals_encoded = encoder.fit_transform(categoricalFeat)

In [7]:
# create test/train set with missing data...
idx_missing = df['count'] < 0

X_train = categoricals_encoded.toarray()[np.array(~idx_missing), :]
X_test = categoricals_encoded.toarray()[np.array(idx_missing), :]

In [8]:
y_train, y_test = df['count'][~idx_missing], df['count'][idx_missing]

In [ ]:
print np.min(y_train), np.max(y_train)
print np.min(y_test), np.max(y_test)

0 30
-1 -1


In [ ]:
# now fit a linear model to those matrices!
clf = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')

# fit the model!
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
np.min(y_pred)

In [ ]:
clf.score(X_train, y_train)